In [13]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Pedestrian sensor data file
pedestrian_sensor_locations_path = "../Datasets/pedestrian-counting-system-sensor-locations.csv"

# Read the pedestrian sensor data file
pedestrian_sensor_locations = pd.read_csv(pedestrian_sensor_locations_path)

# Import the OpenWeatherMap API key
#from api_keys import city_of_melbourne_api_key

pedestrian_sensor_locations_df = pd.DataFrame(pedestrian_sensor_locations)
pedestrian_sensor_locations_df

,location_id,Sensor_description,sensor_name,Installation_date,Note,location_type,Status,Direction_1,Direction_2,Latitude,Longitude,Location
0,1,Bourke Street Mall (North),Bou292_T,2009-03-24,NaN,Outdoor,A,East,West,-37.813494,144.965153,"-37.81349441, 144.96515323"
1,3,Melbourne Central,Swa295_T,2009-03-25,NaN,Outdoor,A,North,South,-37.811015,144.964295,"-37.81101524, 144.96429485"
2,7,Birrarung Marr,FedPed_T,2014-12-17,NaN,Outdoor,A,East,West,-37.818629,144.971694,"-37.81862929, 144.97169395"
3,9,Southern Cross Station,Col700_T,2009-03-23,NaN,Outdoor,A,East,West,-37.819830,144.951026,"-37.81982992, 144.95102555"
4,12,New Quay,NewQ_T,2009-01-21,NaN,Outdoor,A,East,West,-37.814580,144.942924,"-37.81457988, 144.94292398"
...,...,...,...,...,...,...,...,...,...,...,...,...
103,111,Harbour Esplanade,HarbBlix_T,2021-03-25,NaN,Outdoor Blix,A,NaN,NaN,-37.814426,144.944366,"-37.81442577, 144.94436558"
104,115,Shrine of rememberance-North-East,ShrinNE_T,2022-10-14,NaN,Outdoor Blix,A,NaN,NaN,-37.829998,144.973768,"-37.82999797, 144.97376842"
105,119,Pop Up Library - Crossbar - Federation Square,PopLibFed_T,2022-01-25,NaN,Indoor,A,In,Out,-37.817882,144.969446,"-37.81788176, 144.96944609"
106,121,Royal Park BBQ near the playground and public ...,RoyBlix_T,2023-01-18,Royal Park BBQ near the playground and public ...,Outdoor Blix,A,NaN,NaN,-37.795052,144.951046,"-37.79505209, 144.95104583"
